In [20]:
import logging
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import requests
import sys
import time
from selenium.webdriver.common.keys import Keys
import urllib.parse as urlparse
from urllib.parse import parse_qs
import pandas as pd
sys.path.append('../src/')
from util.selenium_dispatcher import SeleniumDispatcher
from config import Config

In [3]:
 def get_request_token(kite_url):
        print('Starting to fetch request token for Kite API')
        selenium = SeleniumDispatcher(headless=True)
        driver = selenium.get_driver()
        driver.get(kite_url)
        time.sleep(4)
        username_field = driver.find_element_by_xpath("//input[@type='text']")
        username_field.send_keys(Config.KITE_USER_ID)
        password_field = driver.find_element_by_xpath("//input[@type='password']")
        password_field.send_keys(Config.KITE_PASSWORD)
        password_field.send_keys(Keys.ENTER)
        time.sleep(2)
        pin_field = driver.find_element_by_xpath("//input[@type='password']")
        pin_field.send_keys(Config.KITE_PIN)
        pin_field.send_keys(Keys.ENTER)
        time.sleep(2)
        url = driver.current_url
        parsed = urlparse.urlparse(url)
        print(url)
        token = parse_qs(parsed.query)['request_token'][0]
        print('Request token received!')
        selenium.destroy_driver()
        return token

In [4]:
kite = KiteConnect(api_key=Config.KITE_API_KEY)

In [5]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=37wk1ehie3noonc1&v=3


In [6]:
data = kite.generate_session(get_request_token(kite.login_url()), api_secret=Config.KITE_API_SECRET)

Starting to fetch request token for Kite API
http://localhost/?request_token=Wgb7On6hwLYvfSqpq5sEvO6R37TMDCHI&action=login&status=success
Request token received!


In [7]:
kite.set_access_token(data["access_token"])


In [8]:
instruments = kite.instruments(exchange='NSE')
instruments[0]

OrderedDict([('instrument_token', 258313),
             ('exchange_token', '1009'),
             ('tradingsymbol', 'CPSE INDEX'),
             ('name', 'CPSE INDEX'),
             ('last_price', 0.0),
             ('expiry', ''),
             ('strike', 0.0),
             ('tick_size', 0.0),
             ('lot_size', 0),
             ('instrument_type', 'EQ'),
             ('segment', 'INDICES'),
             ('exchange', 'NSE')])

In [9]:
selected = [f"NSE:{x['tradingsymbol']}" for x in instruments[50:55]]
selected

['NSE:NIFTY100 QUALTY30',
 'NSE:NIFTY200 QUALTY30',
 'NSE:NIFTY50 DIV POINT',
 'NSE:NIFTY50 EQL WGT',
 'NSE:NIFTY50 PR 1X INV']

In [10]:
kite.quote(selected)

{'NSE:NIFTY100 QUALTY30': {'instrument_token': 272393,
  'tradingsymbol': 'NIFTY100 QUALTY30',
  'timestamp': datetime.datetime(2020, 4, 20, 14, 4, 49),
  'last_price': 2245.8,
  'net_change': -13.05,
  'ohlc': {'open': 2283.15, 'high': 2284.25, 'low': 2242, 'close': 2258.85}},
 'NSE:NIFTY200 QUALTY30': {'instrument_token': 265737,
  'tradingsymbol': 'NIFTY200 QUALTY30',
  'timestamp': datetime.datetime(2020, 4, 20, 14, 4, 49),
  'last_price': 8592.4,
  'net_change': -45.95,
  'ohlc': {'open': 8715.95,
   'high': 8718.05,
   'low': 8578.65,
   'close': 8638.35}},
 'NSE:NIFTY50 DIV POINT': {'instrument_token': 265225,
  'tradingsymbol': 'NIFTY50 DIV POINT',
  'timestamp': datetime.datetime(2020, 4, 20, 14, 4, 49),
  'last_price': 0,
  'net_change': -149.1,
  'ohlc': {'open': 0, 'high': 0, 'low': 0, 'close': 149.1}},
 'NSE:NIFTY50 EQL WGT': {'instrument_token': 271625,
  'tradingsymbol': 'NIFTY50 EQL WGT',
  'timestamp': datetime.datetime(2020, 4, 20, 14, 4, 49),
  'last_price': 9492.95,

In [11]:
kite.ohlc(selected)

{'NSE:NIFTY100 QUALTY30': {'instrument_token': 272393,
  'last_price': 2245.75,
  'ohlc': {'open': 2283.15, 'high': 2284.25, 'low': 2242, 'close': 2258.85}},
 'NSE:NIFTY200 QUALTY30': {'instrument_token': 265737,
  'last_price': 8592.25,
  'ohlc': {'open': 8715.95,
   'high': 8718.05,
   'low': 8578.65,
   'close': 8638.35}},
 'NSE:NIFTY50 DIV POINT': {'instrument_token': 265225,
  'last_price': 0,
  'ohlc': {'open': 0, 'high': 0, 'low': 0, 'close': 149.1}},
 'NSE:NIFTY50 EQL WGT': {'instrument_token': 271625,
  'last_price': 9492.9,
  'ohlc': {'open': 9654.75, 'high': 9656.25, 'low': 9473.8, 'close': 9560.7}},
 'NSE:NIFTY50 PR 1X INV': {'instrument_token': 259081,
  'last_price': 476.55,
  'ohlc': {'open': 470.75, 'high': 478.9, 'low': 470.75, 'close': 476.95}}}

In [12]:
kite.ltp(selected)

{'NSE:NIFTY100 QUALTY30': {'instrument_token': 272393, 'last_price': 2245.75},
 'NSE:NIFTY200 QUALTY30': {'instrument_token': 265737, 'last_price': 8592.25},
 'NSE:NIFTY50 DIV POINT': {'instrument_token': 265225, 'last_price': 0},
 'NSE:NIFTY50 EQL WGT': {'instrument_token': 271625, 'last_price': 9492.9},
 'NSE:NIFTY50 PR 1X INV': {'instrument_token': 259081, 'last_price': 476.55}}

In [13]:
# required_instruments = list()

# for instrument in instruments:
#     if 'BANKNIFTY20312' in instrument['tradingsymbol']:
#         required_instruments.append(instrument)

# len(required_instruments)

# df = pd.DataFrame.from_dict(required_instruments)

# df_put = df[df.instrument_type == 'PE'].reset_index().drop('index', axis=1)

# df_call = df[df.instrument_type == 'CE'].reset_index().drop('index', axis=1)

In [14]:
# Initialise
kws = KiteTicker(Config.KITE_API_KEY, data['access_token'])
tick_count = 0

In [16]:
## Get token of required instrument
selected_instrument = ''
for instrument in instruments:
    if instrument['tradingsymbol'] == 'BPCL':
        selected_instrument = instrument['instrument_token']
        break
selected_instrument

134657

In [17]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    global tick_count
    print("Ticks: {}".format(ticks))
    tick_count = tick_count + 1
    print("Length: {}".format(tick_count))
#     if tick_count > 5:
#         ws.close()

def on_connect(ws, response):
    pass
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([134657])
#     ws.subscribe([134657])
    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_LTP, [134657, 408065, 237569, 389377, 2791681])
    ws.set_mode(ws., [134657])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

In [18]:
# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

In [19]:
# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 134657, 'last_price': 368.25, 'last_quantity': 2, 'average_price': 367.55, 'volume': 4792974, 'buy_quantity': 428244, 'sell_quantity': 481886, 'ohlc': {'open': 362.15, 'high': 372.9, 'low': 361.2, 'close': 360.1}, 'change': 2.263260205498466, 'last_trade_time': datetime.datetime(2020, 4, 20, 14, 5, 30), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2020, 4, 20, 14, 5, 31), 'depth': {'buy': [{'quantity': 10, 'price': 368.1, 'orders': 1}, {'quantity': 842, 'price': 368.0, 'orders': 5}, {'quantity': 42, 'price': 367.95, 'orders': 1}, {'quantity': 23, 'price': 367.9, 'orders': 1}, {'quantity': 30, 'price': 367.85, 'orders': 1}], 'sell': [{'quantity': 12, 'price': 368.15, 'orders': 1}, {'quantity': 27, 'price': 368.2, 'orders': 1}, {'quantity': 87, 'price': 368.25, 'orders': 1}, {'quantity': 3435, 'price': 368.3, 'orders': 1}, {'quantity': 219, 'price': 368.35, 'orders': 1}]}}]
Length: 1
Ticks: [{'tr

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "/Users/mayank.gupta/Moneygen/nseAPI/env/lib/python3.7/site-packages/twisted/internet/base.py", line 706, in disconnectAll
    failure.Failure(main.CONNECTION_LOS

Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 134657, 'last_price': 368.25, 'last_quantity': 1, 'average_price': 367.55, 'volume': 4798154, 'buy_quantity': 431053, 'sell_quantity': 475696, 'ohlc': {'open': 362.15, 'high': 372.9, 'low': 361.2, 'close': 360.1}, 'change': 2.263260205498466, 'last_trade_time': datetime.datetime(2020, 4, 20, 14, 5, 43), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2020, 4, 20, 14, 5, 43), 'depth': {'buy': [{'quantity': 341, 'price': 368.25, 'orders': 4}, {'quantity': 92, 'price': 368.2, 'orders': 2}, {'quantity': 529, 'price': 368.15, 'orders': 4}, {'quantity': 219, 'price': 368.1, 'orders': 1}, {'quantity': 172, 'price': 368.05, 'orders': 3}], 'sell': [{'quantity': 136, 'price': 368.6, 'orders': 4}, {'quantity': 220, 'price': 368.65, 'orders': 3}, {'quantity': 2731, 'price': 368.7, 'orders': 10}, {'quantity': 1129, 'price': 368.75, 'orders': 6}, {'quantity': 1939, 'price': 368.8, 'orders': 4}]}}]
Length: 12
